# Starting project: importing libraries, cleaning data, and removing outliers

## Importing thư viện

In [1]:
# !pip install --upgrade tensorflow

In [2]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [3]:
import warnings
import os
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from keras.optimizers import RMSprop

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,mean_squared_error, mean_absolute_error, r2_score
from keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [4]:
# Bỏ qua các cảnh báo trong tương lai và cảnh báo không dùng nữa để không làm cho sổ ghi chép đầy cảnh báo
warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive

# Kết nối với driver
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
#Đọc 5 dòng đầu tiên của tập dữ liệu
df = pd.read_csv("/content/drive/MyDrive/Học máy/VN_housing_dataset.csv")
df.head()

,Unnamed: 0,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,0.0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,1.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2.0,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,3.0,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,4.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"


In [7]:
# Độ dài của tập dữ liệu trước khi làm sạch và loại bỏ các ngoại lệ
print("The initial length of the dataset is", str(len(df)), "rows.")

The initial length of the dataset is 82497 rows.


## Làm sạch dữ liệu và loại bỏ các giá trị ngoại lệ

### Đổi tên/chuyển đổi tên cột

In [8]:
# Đổi tên/Chuyển ngữ các tên cột
df_renamed = df.rename(columns = {"Ngày":"date", "Địa chỉ":"address", "Quận":"district",
                                  "Huyện":"ward", "Loại hình nhà ở":"type_of_housing",
                                 "Giấy tờ pháp lý":"legal_paper", "Số tầng":"num_floors",
                                 "Số phòng ngủ":"num_bed_rooms", "Diện tích":"squared_meter_area",
                                 "Dài":"length_meter", "Rộng":"width_meter", "Giá/m2":"price_in_million_per_square_meter"})

# Loại bỏ cột 'Unnamed: 0' và các giá trị null
df_renamed = df_renamed.drop("Unnamed: 0", axis = 1)
df_renamed = df_renamed.dropna()
df_renamed = df_renamed.reset_index()

# Độ dài của tập dữ liệu sau khi loại bỏ giá trị null
print("The length of the dataset after dropping null values is", str(len(df_renamed)), "rows.")

The length of the dataset after dropping null values is 11473 rows.


### Làm sạch dữ liệu trong mỗi cột

In [9]:
# Loại bỏ những ngôi nhà có tầng và phòng ngủ "10+" vì không thể định lượng chính xác
df_renamed = df_renamed[df_renamed['num_floors'] != 'Nhiều hơn 10']
df_renamed = df_renamed[df_renamed['num_bed_rooms'] != 'nhiều hơn 10 phòng']

# Làm sạch các cột và chuyển đổi các cột số thành kiểu float
df_renamed['district'] = df_renamed['district'].str.replace('Quận ','').str.strip()
df_renamed['ward'] = df_renamed['ward'].str.replace('Phường ','').str.strip()
df_renamed['num_floors'] = df_renamed['num_floors'].str.strip().astype(float)
df_renamed['num_bed_rooms'] = df_renamed['num_bed_rooms'].str.replace(' phòng','').str.strip().astype(float)
df_renamed['squared_meter_area'] = df_renamed['squared_meter_area'].str.replace(' m²','').str.strip().astype(float)
df_renamed['length_meter'] = df_renamed['length_meter'].str.replace(' m','').str.strip().astype(float)
df_renamed['width_meter'] = df_renamed['width_meter'].str.replace(' m','').str.strip().astype(float)

# Làm sạch và quy đổi toàn bộ giá thành triệu/m2 thay vì tỷ đồng/m2 hay tỷ/m2
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²'), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' tỷ/m²'), 'price_in_million_per_square_meter'].str.replace(' tỷ/m²','').str.replace('.','').str.replace(',','.').astype(float) * 1000
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' triệu/m²', na=False), 'price_in_million_per_square_meter'].str.replace(' triệu/m²','').str.replace(',','.').astype(float)
df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'] = df_renamed.loc[df_renamed['price_in_million_per_square_meter'].str.contains(' đ/m²', na=False), 'price_in_million_per_square_meter'].str.replace(' đ/m²','').str.replace('.','').astype(float) * 0.000001

In [10]:
# Tạo dummies(biến giả) cho các cột phân loại
dummy_type_of_housing = pd.get_dummies(df_renamed.type_of_housing, prefix="housing_type")
dummy_legal_paper = pd.get_dummies(df_renamed.legal_paper, prefix="legal_paper")
dummy_district = pd.get_dummies(df_renamed.district, prefix="district")
dummy_ward = pd.get_dummies(df_renamed.ward, prefix="ward")

# Kết hợp các cột dummies(biến giả) vào dataframe và loại bỏ các cột không cần thiết
df_cleaned = pd.concat([df_renamed, dummy_type_of_housing, dummy_legal_paper, dummy_district, dummy_ward], axis=1)
df_cleaned = df_cleaned.drop(['index', 'date', 'address', 'district', 'ward', 'type_of_housing', 'legal_paper'], axis = 1)
df_cleaned.head()

,num_floors,num_bed_rooms,squared_meter_area,length_meter,width_meter,price_in_million_per_square_meter,housing_type_Nhà biệt thự,"housing_type_Nhà mặt phố, mặt tiền","housing_type_Nhà ngõ, hẻm",housing_type_Nhà phố liền kề,...,ward_Đại Mỗ,ward_Định Công,ward_Đống Mác,ward_Đồng Mai,ward_Đồng Nhân,ward_Đồng Tâm,ward_Đồng Xuân,ward_Đội Cấn,ward_Đức Giang,ward_Đức Thắng
0,4.0,4.0,40.0,10.0,4.0,65.0,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,5.0,4.0,52.0,12.0,4.2,93.27,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,5.0,5.0,90.0,18.0,5.0,108.89,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,4.0,3.0,32.0,6.6,4.5,60.94,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,3.0,2.0,42.0,11.0,4.0,29.76,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
# In ra các cột và giá trị của 5 dòng đầu tiên sau khi làm sạch
for index, row in df_cleaned.head().iterrows():
    print(", ".join([f"{col}: {row[col]}" for col in df_cleaned.columns]))

num_floors: 4.0, num_bed_rooms: 4.0, squared_meter_area: 40.0, length_meter: 10.0, width_meter: 4.0, price_in_million_per_square_meter: 65.0, housing_type_Nhà biệt thự: False, housing_type_Nhà mặt phố, mặt tiền: False, housing_type_Nhà ngõ, hẻm: True, housing_type_Nhà phố liền kề: False, legal_paper_Giấy tờ khác: False, legal_paper_Đang chờ sổ: False, legal_paper_Đã có sổ: True, district_Ba Đình: False, district_Bắc Từ Liêm: False, district_Cầu Giấy: False, district_Hai Bà Trưng: True, district_Hoàn Kiếm: False, district_Hoàng Mai: False, district_Huyện Chương Mỹ: False, district_Huyện Gia Lâm: False, district_Huyện Hoài Đức: False, district_Huyện Mê Linh: False, district_Huyện Phúc Thọ: False, district_Huyện Quốc Oai: False, district_Huyện Sóc Sơn: False, district_Huyện Thanh Oai: False, district_Huyện Thanh Trì: False, district_Huyện Thường Tín: False, district_Huyện Thạch Thất: False, district_Huyện Đan Phượng: False, district_Huyện Đông Anh: False, district_Hà Đông: False, district

### Loại bỏ các giá trị ngoại lai bằng phương pháp IQR

In [12]:
#Hàm loại bỏ ngoại lệ sử dụng phương pháp IQR
def remove_outlier_IQR(df, series):
    Q1=df[series].quantile(0.25)
    Q3=df[series].quantile(0.75)
    IQR=Q3-Q1
    df_final=df[~((df[series]<(Q1-1.5*IQR)) | (df[series]>(Q3+1.5*IQR)))]
    return df_final

# Loại bỏ ngoại lệ trong các cột được chỉ định
removed_outliers = df_cleaned
columns_to_remove_outliers = ['num_floors', 'num_bed_rooms', 'squared_meter_area', 'length_meter',
                              'width_meter', 'price_in_million_per_square_meter']
for column in columns_to_remove_outliers:
    removed_outliers = remove_outlier_IQR(removed_outliers, column)

print("The final length of the dataset is", str(len(removed_outliers)), "rows.")

The final length of the dataset is 7328 rows.


## Chuẩn hóa các biến sử dụng Standard Scaler và tạo tập train và test

In [13]:
housing = removed_outliers

# Chia dữ liệu thành biến dự đoán (X) và biến phản hồi (y)
X = housing.loc[:, housing.columns != 'price_in_million_per_square_meter']
y = housing[['price_in_million_per_square_meter']]
to_be_scaled = ['num_floors', 'num_bed_rooms', 'squared_meter_area', 'length_meter', 'width_meter']

# Bắt đầu chia tỷ lệ
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

X_scaled = X
y_scaled = y

# Lưu trữ đối tượng phù hợp để tham khảo và đảo ngược tỷ lệ sau này
PredictorScalerFit=PredictorScaler.fit(X_scaled[to_be_scaled])
TargetVarScalerFit=TargetVarScaler.fit(y_scaled)

# Tạo các giá trị tiêu chuẩn của X và y
X_scaled[to_be_scaled]=PredictorScalerFit.transform(X_scaled[to_be_scaled])
y_scaled=TargetVarScalerFit.transform(y)

# Chuyển đổi dữ liệu thành mảng numpy
X_array = np.array(X_scaled.values).astype("float32")
y_array = np.array(y_scaled).astype("float32")

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=2032)

# Chuyển đổi các mảng numpy trở lại DataFrame để kiểm tra các cột
X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_test_df = pd.DataFrame(X_test, columns=X.columns)
y_train_df = pd.DataFrame(y_train, columns=y.columns)
y_test_df = pd.DataFrame(y_test, columns=y.columns)

# Kiểm tra các cột và một vài hàng đầu tiên
print("X_train columns:")
print(X_train_df.columns)
# print(X_train_df.head())



# Kiểm tra độ chính xác để xem liệu tất cả các mảng huấn luyện và thử nghiệm có kích thước chính xác hay không
if X_train.shape[0] == y_train.shape[0] and X_train.shape[1] == X_test.shape[1] and X_test.shape[0] == y_test.shape[0] and y_train.shape[1] == y_test.shape[1]:
    print("All train and test sets have correct dimensions.")

X_train columns:
Index(['num_floors', 'num_bed_rooms', 'squared_meter_area', 'length_meter',
       'width_meter', 'housing_type_Nhà biệt thự',
       'housing_type_Nhà mặt phố, mặt tiền', 'housing_type_Nhà ngõ, hẻm',
       'housing_type_Nhà phố liền kề', 'legal_paper_Giấy tờ khác',
       ...
       'ward_Đại Mỗ', 'ward_Định Công', 'ward_Đống Mác', 'ward_Đồng Mai',
       'ward_Đồng Nhân', 'ward_Đồng Tâm', 'ward_Đồng Xuân', 'ward_Đội Cấn',
       'ward_Đức Giang', 'ward_Đức Thắng'],
      dtype='object', length=280)
All train and test sets have correct dimensions.


# Training a Artificial Neural Network (ANN) for Regression

## Training the ANN model with the best parameters

In [14]:
import numpy as np
import pandas as pd

# Hàm kích hoạt ReLU
def relu(x):
  return np.maximum(0, x)

# Đạo hàm của hàm kích hoạt ReLU
def relu_derivative(x):
  return np.where(x > 0, 1, 0)

# Hàm mất mát MSE (Mean Squared Error)
def mse(y_true, y_pred):
  return np.mean(np.square(y_true - y_pred))#tính trung bình của bình phương sai số giữa giá trị thực và giá trị dự đoán.

# Đạo hàm của hàm mất mát MSE
def mse_derivative(y_true, y_pred):
  return 2 * (y_pred - y_true) / y_true.size#tính toán gradient của MSE đối với giá trị dự đoán.

# Khởi tạo trọng số ngẫu nhiên
def initialize_weights(input_size, output_size):
  return np.random.randn(input_size, output_size) * 0.01#Khởi tạo trọng số ngẫu nhiên với phân phối chuẩn có giá trị nhỏ, để làm cho mạng nơ-ron bắt đầu học từ một điểm gần với 0.

# Lớp Dense
class Dense:
  def __init__(self, input_size, output_size):
    self.weights = initialize_weights(input_size, output_size)
    self.biases = np.zeros((1, output_size))

  def forward(self, inputs):#Tính toán đầu ra của lớp với inputs.
    self.inputs = inputs
    return np.dot(inputs, self.weights) + self.biases

  def backward(self, dvalues, learning_rate):#Tính toán gradient và cập nhật trọng số và biases trong quá trình lan truyền ngược.
    # Tính gradient
    dweights = np.dot(self.inputs.T, dvalues)
    dbiases = np.sum(dvalues, axis=0, keepdims=True)
    # Cập nhật trọng số và biases
    self.weights -= learning_rate * dweights
    self.biases -= learning_rate * dbiases
    # Truyền gradient ngược về lớp trước
    dinputs = np.dot(dvalues, self.weights.T)
    return dinputs

# Lớp RMSprop optimizer
class RMSprop:
  def __init__(self, learning_rate=0.001, momentum=0.9):
    self.learning_rate = learning_rate
    self.momentum = momentum
    self.s = None

  def update(self, weights, dweights):#Cập nhật trọng số dựa trên gradient và giá trị s (dùng để điều chỉnh tốc độ học).
    # Khởi tạo s nếu chưa có
    if self.s is None:
      self.s = np.zeros_like(weights)
    # Cập nhật s
    self.s = self.momentum * self.s + (1 - self.momentum) * np.square(dweights)
    # Cập nhật trọng số
    weights -= self.learning_rate * dweights / (np.sqrt(self.s) + 1e-7)
    return weights

# Mô hình ANN
class ANN:
  def __init__(self):
    self.layers = []
    self.optimizer = None

 # Thêm một lớp vào mô hình
  def add(self, layer):
    self.layers.append(layer)

  # Biên dịch mô hình với thuật toán tối ưu
  def compile(self, optimizer):
    self.optimizer = optimizer

  # Huấn luyện mô hình
  def fit(self, X, y, epochs, batch_size):
    for epoch in range(epochs):
      for batch_start in range(0, len(X), batch_size):
        # Lấy batch dữ liệu
        X_batch = X[batch_start:batch_start + batch_size]
        y_batch = y[batch_start:batch_start + batch_size]
        # Lan truyền xuôi
        output = X_batch
        for layer in self.layers:
          output = layer.forward(output)
        # Tính loss
        loss = mse(y_batch, output)
        # Lan truyền ngược
        dvalues = mse_derivative(y_batch, output)
        for layer in reversed(self.layers):
          dvalues = layer.backward(dvalues, self.optimizer.learning_rate)
      # In loss sau mỗi epoch
      print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

  # Hàm dự đoán
  def predict(self, X):
    output = X
    for layer in self.layers:
      output = layer.forward(output)
    return output

# Khởi tạo mô hình ANN
ANN = ANN()
ANN.add(Dense(input_size=X_train.shape[1], output_size=10))
ANN.add(Dense(input_size=10, output_size=1))
ANN.compile(optimizer=RMSprop(learning_rate=0.001, momentum=0.9))

# Huấn luyện mô hình
ANN.fit(X_train, y_train, epochs=50, batch_size=30)

# Dự đoán trên tập kiểm tra
ANN_predictions = ANN.predict(X_test)

# Chuyển đổi dữ liệu dự đoán giá về thang giá gốc
ANN_predictions = TargetVarScalerFit.inverse_transform(ANN_predictions)

# Chuyển đổi dữ liệu giá y_test về thang giá gốc
y_test_orig = TargetVarScalerFit.inverse_transform(y_test)

# Chuyển đổi dữ liệu test về thang gốc
Test_Data = np.concatenate((PredictorScalerFit.inverse_transform(X_test[:,:5]), X_test[:,5:]), axis=1)

#  Tái tạo lại tập dữ liệu, bây giờ với giá được dự đoán bằng mô hình ANN
TestingData = pd.DataFrame(data=Test_Data, columns=X.columns)
TestingData['Price'] = y_test_orig
TestingData['ANN_predictions'] = ANN_predictions


import joblib

# # Lưu mô hình RF đã huấn luyện vào file
# joblib.dump(ANN, '/content/drive/MyDrive/Học máy/ANN_model.pkl')


Epoch 1/50, Loss: 0.9844
Epoch 2/50, Loss: 0.9831
Epoch 3/50, Loss: 0.9823
Epoch 4/50, Loss: 0.9818
Epoch 5/50, Loss: 0.9815
Epoch 6/50, Loss: 0.9813
Epoch 7/50, Loss: 0.9813
Epoch 8/50, Loss: 0.9814
Epoch 9/50, Loss: 0.9816
Epoch 10/50, Loss: 0.9819
Epoch 11/50, Loss: 0.9824
Epoch 12/50, Loss: 0.9831
Epoch 13/50, Loss: 0.9843
Epoch 14/50, Loss: 0.9860
Epoch 15/50, Loss: 0.9884
Epoch 16/50, Loss: 0.9914
Epoch 17/50, Loss: 0.9949
Epoch 18/50, Loss: 0.9981
Epoch 19/50, Loss: 1.0004
Epoch 20/50, Loss: 1.0010
Epoch 21/50, Loss: 0.9992
Epoch 22/50, Loss: 0.9947
Epoch 23/50, Loss: 0.9877
Epoch 24/50, Loss: 0.9782
Epoch 25/50, Loss: 0.9667
Epoch 26/50, Loss: 0.9536
Epoch 27/50, Loss: 0.9393
Epoch 28/50, Loss: 0.9242
Epoch 29/50, Loss: 0.9085
Epoch 30/50, Loss: 0.8924
Epoch 31/50, Loss: 0.8762
Epoch 32/50, Loss: 0.8601
Epoch 33/50, Loss: 0.8442
Epoch 34/50, Loss: 0.8285
Epoch 35/50, Loss: 0.8134
Epoch 36/50, Loss: 0.7987
Epoch 37/50, Loss: 0.7847
Epoch 38/50, Loss: 0.7714
Epoch 39/50, Loss: 0.

# Training a DecisionTree model for Regression

In [23]:
import numpy as np
import os
from sklearn.metrics import mean_squared_error

# Tắt các thông báo và cảnh báo của TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_SETTINGS"] = "false"
# Lớp Node đại diện cho một nút trong cây quyết định.
class Node:
    def __init__(self, feature_index=None, threshold=None, value=None, left=None, right=None):
        self.feature_index = feature_index  # Chỉ mục của thuộc tính (feature) sẽ được sử dụng cho node này
        self.threshold = threshold  # Ngưỡng (threshold) để phân tách dữ liệu
        self.value = value  # Giá trị dự đoán tại node lá (leaf node)
        self.left = left  # Node con bên trái
        self.right = right  # Node con bên phải

#  Lớp DecisionTreeRegressor xây dựng cây quyết định để dự đoán.
class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth  # Độ sâu tối đa của cây
        self.min_samples_split = min_samples_split  # Số mẫu tối thiểu cần để phân chia một node

# Huấn luyện cây quyết định trên tập dữ liệu X và y.
    def fit(self, X, y):
        self.n_features = X.shape[1] # Số lượng thuộc tính
        self.tree_ = self._grow_tree(X, y)# Xây dựng cây quyết định

#  Xây dựng cây quyết định.
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape# Lấy số lượng mẫu và số lượng thuộc tính
        # Tính toán giá trị dự đoán cho node hiện tại
        value = np.mean(y)

        # Điều kiện dừng: nếu đạt độ sâu tối đa hoặc số lượng mẫu ít hơn ngưỡng tối thiểu
        if (self.max_depth is not None and depth >= self.max_depth) or n_samples < self.min_samples_split:
            return Node(value=value)

        # Tìm phân chia tốt nhất
        best_split = self._best_split(X, y, n_samples, n_features)

        if best_split is None:# Nếu không tìm được phân chia tốt nhất, trả về node lá
            return Node(value=value)

        feature_index, threshold, X_left, X_right, y_left, y_right = best_split

 # Đệ quy xây dựng cây con bên trái và bên phải
        left = self._grow_tree(X_left, y_left, depth + 1)
        right = self._grow_tree(X_right, y_right, depth + 1)
 # Trả về node hiện tại với các thông tin về phân chia và cây con
        return Node(feature_index, threshold, value, left, right)

# Tìm phân chia tốt nhất cho node hiện tại.
    def _best_split(self, X, y, n_samples, n_features):
        best_idx, best_thr = None, None# Khởi tạo chỉ mục và ngưỡng tốt nhất
        best_mse = float('inf')# Khởi tạo MSE tốt nhất là vô cùng lớn
        for feature_idx in range(n_features):# Duyệt qua từng thuộc tính
            thresholds, unique_idxs = np.unique(X[:, feature_idx], return_index=True)# Lấy các giá trị ngưỡng duy nhất
            for idx in unique_idxs: # Duyệt qua từng ngưỡng duy nhất
                threshold = X[idx, feature_idx]# Lấy giá trị ngưỡng
                y_left = y[X[:, feature_idx] <= threshold]# Dữ liệu nhãn bên trái
                y_right = y[X[:, feature_idx] > threshold] # Dữ liệu nhãn bên phải
                if len(y_left) > 0 and len(y_right) > 0:# Kiểm tra nếu có dữ liệu cả hai bên
                    mse = self._calculate_mse(y_left, y_right)  # Tính toán MSE cho phân chia hiện tại
                    if mse < best_mse:  # Nếu MSE tốt hơn, cập nhật chỉ mục và ngưỡng tốt nhất
                        best_idx, best_thr = feature_idx, threshold
                        best_mse = mse
       # Nếu không tìm được chỉ mục tốt nhất, trả về None
        if best_idx is None:
            return None

      # Tạo dữ liệu bên trái và bên phải sau khi phân chia
        X_left = X[X[:, best_idx] <= best_thr]
        X_right = X[X[:, best_idx] > best_thr]
        y_left = y[X[:, best_idx] <= best_thr]
        y_right = y[X[:, best_idx] > best_thr]

        return best_idx, best_thr, X_left, X_right, y_left, y_right

    def _calculate_mse(self, y_left, y_right):#Tính toán MSE cho một phân chia cụ thể
        return np.mean(np.square(y_left - np.mean(y_left))) + np.mean(np.square(y_right - np.mean(y_right)))

    def predict(self, X): #Dự đoán giá trị cho tập dữ liệu X.
        return np.array([self._predict_tree(x, self.tree_) for x in X])

    def _predict_tree(self, x, tree): #Dự đoán giá trị cho một mẫu x sử dụng cây quyết định.
        if tree.left is None and tree.right is None:
            return tree.value
        if x[tree.feature_index] <= tree.threshold:
            return self._predict_tree(x, tree.left)
        return self._predict_tree(x, tree.right)

# Khởi tạo mô hình Decision Tree với các tham số tốt nhất
DT_best_params = {
    'max_depth': 100,
    'min_samples_split': 10,
    'min_samples_leaf': 2
}



# Tạo một mô hình DecisionTreeRegressor với các tham số tốt nhất
tree_model = DecisionTreeRegressor(max_depth=DT_best_params['max_depth'], min_samples_split=DT_best_params['min_samples_split'])

# Huấn luyện mô hình Decision Tree với các tham số tốt nhất
tree_model.fit(X_train, y_train)

# Dự đoán và tính toán loss trên tập kiểm tra
tree_predictions = tree_model.predict(X_test)
tree_predictions = tree_predictions.reshape(-1, 1)

# Chuyển đổi dữ liệu giá đã dự đoán trở lại thang đo giá gốc
tree_predictions = TargetVarScalerFit.inverse_transform(tree_predictions)
# Chuyển đổi dữ liệu giá y_test về thang giá gốc
y_test_orig = TargetVarScalerFit.inverse_transform(y_test)
# Chuyển đổi dữ liệu test về thang gốc
Test_Data = np.concatenate((PredictorScalerFit.inverse_transform(X_test[:,:5]), X_test[:,5:]), axis=1)

# Tính toán MSE chuẩn hóa
tree_loss = mean_squared_error(y_test_orig, tree_predictions)
price_range = y_test_orig.max() - y_test_orig.min()
normalized_mse = tree_loss / (price_range ** 2)

print("Loss:", normalized_mse)

# # Thêm các dự đoán của cây vào dữ liệu kiểm tra
# TestingData = pd.DataFrame(data=Test_Data, columns=X.columns)
# TestingData['Price'] = y_test_orig
TestingData['Tree_predictions'] = tree_predictions




# # Lưu mô hình RF đã huấn luyện vào file
# joblib.dump(tree_model, '/content/drive/MyDrive/Học máy/dt_model.pkl')


Loss: 0.02648517159064487


# Training a Random Forest model for Regression

In [26]:
import numpy as np
import os
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample

# Tắt các thông báo và cảnh báo của TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["KMP_SETTINGS"] = "false"

class Node:
    def __init__(self, feature_index=None, threshold=None, value=None, left=None, right=None):
        self.feature_index = feature_index # chỉ số của feature để chia cây
        self.threshold = threshold # ngưỡng để chia cây
        self.value = value# giá trị dự đoán tại node lá
        self.left = left# node con bên trái
        self.right = right# node con bên phải

#  Lớp DecisionTreeRegressor xây dựng cây quyết định để dự đoán.
class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth # độ sâu tối đa của cây
        self.min_samples_split = min_samples_split # số lượng mẫu tối thiểu để chia node

# Huấn luyện cây quyết định trên tập dữ liệu X và y.
    def fit(self, X, y):
        self.n_features = X.shape[1]# số lượng feature
        self.tree_ = self._grow_tree(X, y)# xây dựng cây quyết định từ dữ liệu huấn luyện

#  Xây dựng cây quyết định.
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape# Lấy số lượng mẫu và số lượng thuộc tính
        value = np.mean(y)# giá trị dự đoán tại node hiện tại là trung bình các giá trị y

# Điều kiện dừng: nếu đạt độ sâu tối đa hoặc số lượng mẫu ít hơn ngưỡng tối thiểu
        if (self.max_depth is not None and depth >= self.max_depth) or n_samples < self.min_samples_split:
            return Node(value=value)

# Tìm phân chia tốt nhất
        best_split = self._best_split(X, y, n_samples, n_features)

        if best_split is None:
            return Node(value=value)

        feature_index, threshold, X_left, X_right, y_left, y_right = best_split
# Đệ quy xây dựng cây con bên trái và bên phải
        left = self._grow_tree(X_left, y_left, depth + 1)
        right = self._grow_tree(X_right, y_right, depth + 1)
 # Trả về node hiện tại với các thông tin về phân chia và cây con
        return Node(feature_index, threshold, value, left, right)

# Tìm phân chia tốt nhất cho node hiện tại.
    def _best_split(self, X, y, n_samples, n_features):
        best_idx, best_thr = None, None# Khởi tạo chỉ mục và ngưỡng tốt nhất
        best_mse = float('inf')# Khởi tạo MSE tốt nhất là vô cùng lớn
        for feature_idx in range(n_features):# Duyệt qua từng thuộc tính
            thresholds, unique_idxs = np.unique(X[:, feature_idx], return_index=True)# Lấy các giá trị ngưỡng duy nhất
            for idx in unique_idxs:# Duyệt qua từng ngưỡng duy nhất
                threshold = X[idx, feature_idx]# Lấy giá trị ngưỡng
                y_left = y[X[:, feature_idx] <= threshold]# Dữ liệu nhãn bên trái
                y_right = y[X[:, feature_idx] > threshold]# Dữ liệu nhãn bên phải
                if len(y_left) > 0 and len(y_right) > 0:# Kiểm tra nếu có dữ liệu cả hai bên
                    mse = self._calculate_mse(y_left, y_right) # Tính toán MSE cho phân chia hiện tại
                    if mse < best_mse:# Nếu MSE tốt hơn, cập nhật chỉ mục và ngưỡng tốt nhất
                        best_idx, best_thr = feature_idx, threshold
                        best_mse = mse

   # Nếu không tìm được chỉ mục tốt nhất, trả về None
        if best_idx is None:
            return None

  # Tạo dữ liệu bên trái và bên phải sau khi phân chia
        X_left = X[X[:, best_idx] <= best_thr]
        X_right = X[X[:, best_idx] > best_thr]
        y_left = y[X[:, best_idx] <= best_thr]
        y_right = y[X[:, best_idx] > best_thr]

        return best_idx, best_thr, X_left, X_right, y_left, y_right

    def _calculate_mse(self, y_left, y_right):#Tính toán MSE cho một phân chia cụ thể
        return np.mean(np.square(y_left - np.mean(y_left))) + np.mean(np.square(y_right - np.mean(y_right)))

    def predict(self, X):#Dự đoán giá trị cho tập dữ liệu X.
        return np.array([self._predict_tree(x, self.tree_) for x in X])

    def _predict_tree(self, x, tree): #Dự đoán giá trị cho một mẫu x sử dụng cây quyết định.
        if tree.left is None and tree.right is None:
            return tree.value
        if x[tree.feature_index] <= tree.threshold:
            return self._predict_tree(x, tree.left)
        return self._predict_tree(x, tree.right)

class RandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2):
        self.n_estimators = n_estimators# số lượng cây trong rừng
        self.max_depth = max_depth# độ sâu tối đa của mỗi cây quyết định
        self.min_samples_split = min_samples_split# số lượng mẫu tối thiểu để chia một node trong cây quyết định
        self.trees = []  # danh sách các cây quyết định trong rừng

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            X_sample, y_sample = resample(X, y)# lấy mẫu ngẫu nhiên từ dữ liệu huấn luyện
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)# huấn luyện cây quyết định trên mẫu dữ liệu
            self.trees.append(tree) # thêm cây đã huấn luyện vào danh sách cây trong rừng

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])# dự đoán của từng cây trong rừng
        return np.mean(tree_preds, axis=0) # trung bình các dự đoán để ra kết quả cuối cùng

# Khởi tạo mô hình RandomForest với các tham số tốt nhất
RF_best_params = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2
}

# Tạo một mô hình RandomForestRegressor với các tham số tốt nhất
forest_model = RandomForestRegressor(n_estimators=RF_best_params['n_estimators'],
                                     max_depth=RF_best_params['max_depth'],
                                     min_samples_split=RF_best_params['min_samples_split'])

# Huấn luyện mô hình Random Forest với các tham số tốt nhất
forest_model.fit(X_train, y_train)

# Dự đoán và tính toán loss trên tập kiểm tra
forest_predictions = forest_model.predict(X_test)
forest_predictions = forest_predictions.reshape(-1, 1)

# Chuyển đổi dữ liệu giá đã dự đoán trở lại thang đo giá gốc
forest_predictions = TargetVarScalerFit.inverse_transform(forest_predictions)
# Chuyển đổi dữ liệu giá y_test về thang giá gốc
y_test_orig = TargetVarScalerFit.inverse_transform(y_test)
# Chuyển đổi dữ liệu test về thang gốc
Test_Data = np.concatenate((PredictorScalerFit.inverse_transform(X_test[:,:5]), X_test[:,5:]), axis=1)

# Tính toán MSE chuẩn hóa
forest_loss = mean_squared_error(y_test_orig, forest_predictions)
price_range = y_test_orig.max() - y_test_orig.min()
normalized_mse = forest_loss / (price_range ** 2)

print("Loss:", normalized_mse)

# Thêm các dự đoán của Random Forest vào dữ liệu kiểm tra
TestingData['Forest_predictions'] = forest_predictions

# # Lưu mô hình RF đã huấn luyện vào file
# joblib.dump(forest_model, '/content/drive/MyDrive/Học máy/rf_model.pkl')


Loss: 0.03552024783684903


# Final result: Artifical Neural Network, DecisionTree, Random Forest

In [19]:
# Định nghĩa một hàm đánh giá các dự đoán
def Accuracy_Score(orig, pred):
    MAPE = np.mean(100 * (np.abs(orig - pred) / orig))
    return(100-MAPE)

In [20]:
# Hiển thị điểm số cho cả mô hình ANN và RF
print("Accuracy for the ANN model is:", str(Accuracy_Score(TestingData['Price'], TestingData['ANN_predictions'])))
print("Accuracy for the DT model is:", str(Accuracy_Score(TestingData['Price'], TestingData['Tree_predictions'])))
print("Accuracy for the RF model is:", str(Accuracy_Score(TestingData['Price'], TestingData['Forest_predictions'])))

Accuracy for the ANN model is: 80.06927977278932
Accuracy for the DT model is: 79.55044937133789
Accuracy for the RF model is: 73.45713233947754


#Dự đoán giá nhà

In [22]:
import joblib
import sklearn

# Hàm để định dạng số tiền với dấu phân cách ngăn cách hàng nghìn
def format_currency(amount):
    return "{:,.0f}".format(amount)
# Lưu mô hình RF đã huấn luyện vào file bằng pickle
# Tải mô hình từ file
# RF_loaded = joblib.load('/content/drive/MyDrive/Tài liệu, bài tập của các năm học/Tài liệu học tập kỳ 6/Đồ án Chuyên ngành 1/new_rf_model.pkl')
#
# Ví dụ dữ liệu mới (bạn cần chuẩn bị dữ liệu mới theo cùng định dạng với dữ liệu huấn luyện)
# Dưới đây chỉ là ví dụ, bạn cần thay đổi cho phù hợp với dữ liệu thực tế của bạn
new_data = pd.DataFrame({
    'num_floors': [2],
    'num_bed_rooms': [4],
    'squared_meter_area': [500],
    'length_meter': [25],
    'width_meter': [20],
    "housing_type_Nhà biệt thự": True, "housing_type_Nhà mặt phố, mặt tiền": False, "housing_type_Nhà ngõ, hẻm": False, "housing_type_Nhà phố liền kề": False, "legal_paper_Giấy tờ khác": False, "legal_paper_Đang chờ sổ": False,
    "legal_paper_Đã có sổ": True,
    "district_Ba Đình": True, "district_Bắc Từ Liêm": False, "district_Cầu Giấy": False, "district_Hai Bà Trưng": False, "district_Hoàn Kiếm": False, "district_Hoàng Mai": False, "district_Huyện Chương Mỹ": False, "district_Huyện Gia Lâm": False, "district_Huyện Hoài Đức": False, "district_Huyện Mê Linh": False, "district_Huyện Phúc Thọ": False, "district_Huyện Quốc Oai": False, "district_Huyện Sóc Sơn": False, "district_Huyện Thanh Oai": False, "district_Huyện Thanh Trì": False, "district_Huyện Thường Tín": False, "district_Huyện Thạch Thất": False, "district_Huyện Đan Phượng": False, "district_Huyện Đông Anh": False, "district_Hà Đông": False, "district_Long Biên": False, "district_Nam Từ Liêm": False, "district_Thanh Xuân": False, "district_Thị xã Sơn Tây": False, "district_Tây Hồ": False, "district_Đống Đa": False, "ward_Biên Giang": False, "ward_Bách Khoa": False, "ward_Bùi Thị Xuân": False, "ward_Bưởi": False, "ward_Bạch Mai": False, "ward_Bạch Đằng": False, "ward_Bồ Đề": False, "ward_Chương Dương": False, "ward_Cát Linh": False, "ward_Cầu Diễn": False, "ward_Cầu Dền": False, "ward_Cống Vị": False, "ward_Cổ Nhuế 1": False, "ward_Cổ Nhuế 2": False, "ward_Cửa Nam": False, "ward_Cửa Đông": False, "ward_Cự Khối": False, "ward_Dương Nội": False, "ward_Dịch Vọng": False, "ward_Dịch Vọng Hậu": False, "ward_Gia Thụy": False, "ward_Giang Biên": False, "ward_Giáp Bát": False, "ward_Giảng Võ": False, "ward_Hoàng Liệt": False, "ward_Hoàng Văn Thụ": False, "ward_Hà Cầu": False, "ward_Hàng Buồm": False, "ward_Hàng Bài": False, "ward_Hàng Bông": False, "ward_Hàng Bồ": False, "ward_Hàng Bột": False, "ward_Hàng Gai": False, "ward_Hàng Mã": False, "ward_Hàng Trống": False, "ward_Hàng Đào": False, "ward_Hạ Đình": False, "ward_Khâm Thiên": False, "ward_Khương Mai": False, "ward_Khương Thượng": False, "ward_Khương Trung": False, "ward_Khương Đình": False, "ward_Kim Giang": False, "ward_Kim Liên": False, "ward_Kim Mã": False, "ward_Kiến Hưng": False, "ward_La Khê": False, "ward_Liên Mạc": False, "ward_Liễu Giai": False, "ward_Long Biên": False, "ward_Láng Hạ": False, "ward_Láng Thượng": False,
    "ward_Lê Đại Hành": True, "ward_Lý Thái Tổ": False, "ward_Lĩnh Nam": False, "ward_Mai Dịch": False, "ward_Mai Động": False, "ward_Minh Khai": False, "ward_Mễ Trì": False, "ward_Mộ Lao": False, "ward_Mỹ Đình 1": False, "ward_Mỹ Đình 2": False, "ward_Nam Đồng": False, "ward_Nghĩa Tân": False, "ward_Nghĩa Đô": False, "ward_Nguyễn Du": False, "ward_Nguyễn Trung Trực": False, "ward_Nguyễn Trãi": False, "ward_Ngã Tư Sở": False, "ward_Ngô Quyền": False, "ward_Ngô Thì Nhậm": False, "ward_Ngọc Hà": False, "ward_Ngọc Khánh": False, "ward_Ngọc Lâm": False, "ward_Ngọc Thụy": False, "ward_Nhân Chính": False, "ward_Nhật Tân": False, "ward_Phan Chu Trinh": False, "ward_Phú Diễn": False, "ward_Phú La": False, "ward_Phú Lãm": False, "ward_Phú Lương": False, "ward_Phú Thượng": False, "ward_Phú Thịnh": False, "ward_Phú Đô": False, "ward_Phúc Diễn": False, "ward_Phúc La": False, "ward_Phúc Lợi": False, "ward_Phúc Tân": False, "ward_Phúc Xá": False, "ward_Phúc Đồng": False, "ward_Phương Canh": False, "ward_Phương Liên": False, "ward_Phương Liệt": False, "ward_Phương Mai": False, "ward_Phạm Đình Hổ": False, "ward_Phố Huế": False, "ward_Quan Hoa": False, "ward_Quang Trung": False, "ward_Quán Thánh": False, "ward_Quảng An": False, "ward_Quốc Tử Giám": False, "ward_Quỳnh Lôi": False, "ward_Quỳnh Mai": False, "ward_Sài Đồng": False, "ward_Thanh Lương": False, "ward_Thanh Nhàn": False, "ward_Thanh Trì": False, "ward_Thanh Xuân Bắc": False, "ward_Thanh Xuân Nam": False, "ward_Thanh Xuân Trung": False, "ward_Thành Công": False, "ward_Thượng Cát": False, "ward_Thượng Thanh": False, "ward_Thượng Đình": False, "ward_Thạch Bàn": False, "ward_Thị trấn Chúc Sơn": False, "ward_Thị trấn Phùng": False, "ward_Thị trấn Quang Minh": False, "ward_Thị trấn Sóc Sơn": False, "ward_Thị trấn Trâu Quỳ": False, "ward_Thị trấn Trạm Trôi": False, "ward_Thị trấn Văn Điển": False, "ward_Thị trấn Yên Viên": False, "ward_Thị trấn Đông Anh": False, "ward_Thịnh Liệt": False, "ward_Thịnh Quang": False, "ward_Thổ Quan": False, "ward_Thụy Khuê": False, "ward_Thụy Phương": False, "ward_Trung Hoà": False, "ward_Trung Liệt": False, "ward_Trung Phụng": False, "ward_Trung Tự": False, "ward_Trung Văn": False, "ward_Tràng Tiền": False, "ward_Trúc Bạch": False, "ward_Trương Định": False, "ward_Trần Hưng Đạo": False, "ward_Trần Phú": False, "ward_Tân Mai": False, "ward_Tây Mỗ": False, "ward_Tây Tựu": False, "ward_Tương Mai": False, "ward_Tứ Liên": False, "ward_Việt Hưng": False, "ward_Văn Chương": False, "ward_Văn Miếu": False, "ward_Văn Quán": False, "ward_Vĩnh Hưng": False, "ward_Vĩnh Phúc": False, "ward_Vĩnh Tuy": False, "ward_Vạn Phúc": False, "ward_Xuân La": False, "ward_Xuân Phương": False, "ward_Xuân Tảo": False, "ward_Xuân Đỉnh": False, "ward_Xã An Khánh": False, "ward_Xã An Thượng": False, "ward_Xã Bích Hòa": False, "ward_Xã Bắc Hồng": False, "ward_Xã Cổ Bi": False, "ward_Xã Cổ Đông": False, "ward_Xã Cự Khê": False, "ward_Xã Di Trạch": False, "ward_Xã Duyên Thái": False, "ward_Xã Dương Liễu": False, "ward_Xã Dương Quang": False, "ward_Xã Hoàng Văn Thụ": False, "ward_Xã Hương Ngải": False, "ward_Xã Hải Bối": False, "ward_Xã Hữu Hoà": False, "ward_Xã Khánh Hà": False, "ward_Xã Kim Chung": False, "ward_Xã Kim Hoa": False, "ward_Xã Kim Sơn": False, "ward_Xã Kiêu Kỵ": False, "ward_Xã La Phù": False, "ward_Xã Liên Ninh": False, "ward_Xã Lê Lợi": False, "ward_Xã Mai Lâm": False, "ward_Xã Nam Hồng": False, "ward_Xã Nghĩa Hương": False, "ward_Xã Ngũ Hiệp": False, "ward_Xã Ngọc Hồi": False, "ward_Xã Nhị Khê": False, "ward_Xã Phù Linh": False, "ward_Xã Phù Đổng": False, "ward_Xã Phú Cát": False, "ward_Xã Phú Cường": False, "ward_Xã Phương Trung": False, "ward_Xã Sài Sơn": False, "ward_Xã Sơn Đông": False, "ward_Xã Sơn Đồng": False, "ward_Xã Tam Hiệp": False, "ward_Xã Thanh Liệt": False, "ward_Xã Thượng Mỗ": False, "ward_Xã Thạch Hoà": False, "ward_Xã Tiên Dược": False, "ward_Xã Tiền Phong": False, "ward_Xã Tân Lập": False, "ward_Xã Tân Triều": False, "ward_Xã Tả Thanh Oai": False, "ward_Xã Tứ Hiệp": False, "ward_Xã Vân Canh": False, "ward_Xã Vân Côn": False, "ward_Xã Vân Nội": False, "ward_Xã Võng La": False, "ward_Xã Võng Xuyên": False, "ward_Xã Văn Bình": False, "ward_Xã Vĩnh Quỳnh": False, "ward_Xã Xuân Giang": False, "ward_Xã Yên Thường": False, "ward_Xã Đa Tốn": False, "ward_Xã Đình Xuyên": False, "ward_Xã Đông Dư": False, "ward_Xã Đông Hội": False, "ward_Xã Đông La": False, "ward_Xã Đông Mỹ": False, "ward_Xã Đại Yên": False, "ward_Xã Đại áng": False, "ward_Xã Đặng Xá": False, "ward_Xã Đức Thượng": False, "ward_Yên Hoà": False, "ward_Yên Nghĩa": False, "ward_Yên Phụ": False, "ward_Yên Sở": False, "ward_Yết Kiêu": False, "ward_Ô Chợ Dừa": False, "ward_Điện Biên": False, "ward_Đông Ngạc": False, "ward_Đại Kim": False, "ward_Đại Mỗ": False, "ward_Định Công": False, "ward_Đống Mác": False, "ward_Đồng Mai": False, "ward_Đồng Nhân": False, "ward_Đồng Tâm": False, "ward_Đồng Xuân": False, "ward_Đội Cấn": False, "ward_Đức Giang": False, "ward_Đức Thắng": False


})

# Chuẩn hóa dữ liệu mới
new_data_scaled = new_data.copy()
new_data_scaled[to_be_scaled] = PredictorScalerFit.transform(new_data_scaled[to_be_scaled])

# Chuyển đổi dữ liệu mới thành mảng numpy
new_data_array = np.array(new_data_scaled.values).astype("float32")

# Dự đoán giá sử dụng mô hình RF đã tải
new_predictions = ANN.predict(new_data_array)
new_predictions = new_predictions.reshape(-1, 1)

# Chuyển đổi giá trị dự đoán về đơn vị ban đầu
new_predictions_original = TargetVarScalerFit.inverse_transform(new_predictions)

# Chuyển đổi giá trị dự đoán về VNĐ (tính bằng giả định 1 triệu VNĐ/m2)
price_per_meter = 1_000_000  # Giả sử giá trung bình là 1 triệu VNĐ/m2
new_predictions_vnd = new_predictions_original * new_data['squared_meter_area'].values.reshape(-1, 1) * price_per_meter

# Định dạng số tiền về dạng có dấu phân cách hàng nghìn
formatted_price = format_currency(new_predictions_vnd[0][0])

print("Giá dự đoán của căn nhà là:", formatted_price, "VNĐ")



Giá dự đoán của căn nhà là: 26,923,511,282 VNĐ
